# Data Cleaning Challenge

Thinkful Bootcamp Course

Author: Ian Heaton

Email: iheaton@gmail.com

Mentor: Nemanja Radojkovic

Date: 2017/02/16


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns

Below is the Python code that performs all of the heavy lefting to clean and Impute the Wellcome trust journal data.

In [18]:
PMCregex = r'PMC\d+'
PMIDregex = r'PMID:\s?\d+'


def retrieve_metadata(dataframe):
    """
     Extracts the shape, column names and number of null rows for
     each column of a Pandas data frame. This information is printed
     to standard ouput.

     Args:
        dataframe (dataframe): The Pandas data frame of interest

     Returns:
        Nothing
    """
    num_rows, num_columns = dataframe.shape
    columns = dataframe.columns.values.tolist()
    print('Data frame contains %d columns and %d rows' % (num_columns, num_rows))
    for elem in columns:
        print('column %s has %d empty rows'% (elem, sum(dataframe[elem].isnull())))


def _search_pmcid_regex(id_str):
    """
    Tries to match the string containing the journal ID against
    a PMCID regex. Returns the first instance of a match.

     Args:
        id_str (str): Cell contents of the ‘'PMID/PMCID' column.

     Returns:
       tuple: A tuple containing boolean and string containing
       the new ID.  The boolean may have a value of true indicating
       that a sub string matched the regular expression or false otherwise.
       If false the match str is empty.
    """
    result = False
    match_str = ''
    try:
        match_str = re.search(PMCregex, id_str).group()
        result = True
    except AttributeError:
        pass
    return result, match_str


def _search_pmid_regex(id_str):
    """
     Tries to match the string containing the journal ID against
    a Pub Med regex. Returns the first instance of a match.

     Args:
        id_str (str): Cell contents of the ‘'PMID/PMCID' column.

     Returns:
       tuple: A tuple containing boolean and string containing
       the new ID.  The boolean may have a value of true indicating
       that a sub string matched the regular expression or false otherwise.
       If false the match str is empty.
    """
    result = False
    match_str = ''
    try:
        match_str = re.search(PMIDregex, id_str).group()
        match_str = match_str.replace(':', '')
        match_str = match_str.replace(' ', '')
        result = True
    except AttributeError:
        pass
    return result, match_str


def pmcid_pmid_converter(id_str):
    """
     Specialized data massaging function for Wellcome trust data.
     This function should only be run on the first column containing
     the label ‘PMC ID/PMID’.
     The following assumptions and conversions are carried out;

        1. Those id numbers starting with 3 are PMCID ids
        2. Those id numbers starting with 2 are PMID ids
        3. Those ids without prefix will have either PMIC or PMID appended
        4. NA values will be kept
        5. Any other text in a row that does not adherer to the above two
           forms will be dropped and replaced with NA.
        6. Cells that have both PMIC and PMID identifiers will have
           the 2nd id dropped. Only one id is necessary.

     Args:
        id_str (str): String instance containing a cells entire text.

     Returns:
        str: string instance containing either NA, PMCID* or PMID*
    """
    data = str(id_str)
    data = data.strip('\n')
    has_pmcid_match = False
    has_pmid_match = False
    match_pmcid_str = ''
    match_pmid_str = ''
    result_str = ''

    has_pmcid_match, match_pmcid_str = _search_pmcid_regex(data)
    has_pmid_match, match_pmid_str = _search_pmid_regex(data)

    if has_pmcid_match or has_pmid_match:
        if has_pmcid_match:
            result_str = match_pmcid_str
        elif has_pmid_match:
            result_str = match_pmid_str
    elif data.isdigit():
        if data[0] == '2':
            result_str = 'PMID' + data
        elif data[0] == '3':
            result_str = 'PMCID' + data
    else:
        result_str = 'NA'

    return result_str


def remove_money_symbols(string):
    """
    Removes the pound symbol and trailing dollar sign symbols from
    the Cost data. Resulting data is converted to type float.

    Args:
        string (str): string instance housing cell contents.

    Returns:
        float: Cost of journal article represented as a float
    """
    string = string.replace('$', '')
    return float(string[1:])


def remove_cost_outliers(cost, dataf, threshold):
    """
    Sets any cost values that are equal to or greater than threshold
    to the 90th  quartile for the coat column.
    Assumes that a Seres object is passed in as the dataf argument
    having a single column.

    Args:
        coat (float):           cost value to be capped.
        dataf (Pandas, Series): Series instance that statistic will be retrieved from.
        threshold (decimal):    Outlier threshold

    Returns:
        float: Resulting cost value
    """
    result = cost
    if cost >= threshold:
        result = dataf.quantile(.9)
    return result

In [7]:
file_path = '/media/ianh/space/ThinkfulData/WelcomeTrust/WELLCOME_APCspend2013_forThinkful.csv'
data_frame = pd.read_csv(file_path)
retrieve_metadata(data_frame)

Data frame contains 5 columns and 2127 rows
column PMID/PMCID has 199 empty rows
column Publisher has 0 empty rows
column Journal title has 1 empty rows
column Article title has 0 empty rows
column COST (�) charged to Wellcome (inc VAT when charged) has 0 empty rows


In [9]:
NewIDColumn_Df = pd.DataFrame({'ID':data_frame['PMID/PMCID'].apply(pmcid_pmid_converter)})
retrieve_metadata(NewIDColumn_Df)

Data frame contains 1 columns and 2127 rows
column ID has 0 empty rows


The cost column is now altered to remove any money symbols and converted to float so that proper statistics can be performed on the column. In addition, another trandformation is applied to cap data to the 90th Quartile if the cost value is greater than 99998. Any journal article costing more than 99 thousand pounds is a mistake.

In [29]:
column_name = 'COST (�) charged to Wellcome (inc VAT when charged)'
NewCost1stPass_Df = pd.DataFrame({'Cost':data_frame[column_name].apply(remove_money_symbols)})
NewCost2ndPass_df = pd.DataFrame({'Cost':NewCost1stPass_Df['Cost'].apply(remove_cost_outliers, args=(NewCost1stPass_Df, 999998))})

In [32]:
NewIDColumn_Df['Cost'] = NewCost2ndPass_df['Cost']
NewIDColumn_Df['Publisher'] = data_frame['Publisher']
NewIDColumn_Df['Journal title'] = data_frame['Journal title']
NewIDColumn_Df

,ID,Cost,Publisher,Journal title
0,NA,0,CUP,Psychological Medicine
1,PMC3679557,2381.04,ACS,Biomacromolecules
2,PMC3506128,642.56,ACS,J Med Chem
3,PMC3646402,669.64,ACS,J Med Chem
4,PMC3601604,685.88,ACS,J Org Chem
5,PMC3579457,2392.2,ACS,Journal of Medicinal Chemistry
6,PMC3709265,2367.95,ACS,Journal of Proteome Research
7,PMC3495574,649.33,ACS,Mol Pharm
8,PMC3780468,1294.59,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology
9,PMC3621575,1294.78,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology
